In [1]:
import pandas as pd
import numpy as np
import janitor as jn

In [2]:
events = pd.read_csv(
    "/Users/samuel.oranyeli/Downloads/results.csv",
    parse_dates=["start", "end"],
).iloc[:, 1:]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/samuel.oranyeli/Downloads/results.csv'

In [ ]:
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=True,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

[array([21793, 16872, 19703, ..., 24135,   117,  9932]),
 array([ 6489, 11410,  8579, ...,  4147, 28165, 18350]),
 array([    0,     1,     2, ..., 29996, 29997, 29998])]

In [ ]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=True,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

10.5 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=False,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

17.5 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
